In [1]:
%load_ext autoreload
%autoreload 2

In [72]:
from model import Model
from prompts import SYSTEM_PROMPT
from agent import ReactAgent
from tools import get_transactions

agent = ReactAgent(Model(model_name="llama-3.1-8b-instant"))


In [73]:
query = "How much did I spend on Uber last week?"

response = agent.run(query)

Last Message:  ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_x2xj', function=Function(arguments='{"query": "Uber", "start_date": "2025-05-01", "end_date": "2025-05-08"}', name='get_transactions'), type='function')])
ToolCall:  [ChatCompletionMessageToolCall(id='call_x2xj', function=Function(arguments='{"query": "Uber", "start_date": "2025-05-01", "end_date": "2025-05-08"}', name='get_transactions'), type='function')]
Last Message:  {'role': 'function', 'name': 'get_transactions', 'content': "[{'txn_date': '2025-05-01', 'merchant': 'Uber', 'amt': 242.5}]"}
Last Message:  ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7qap', function=Function(arguments='{"records": "[{\'txn_date\': \'2025-05-01\', \'merchant\': \'Uber\', \'amt\': 242.5}]", "agg": 

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'I apologize, but it seems like there was an error processing your request. Let me try again.\n\n<function=get_transactions>{"query": "Uber", "start_date": "2025-05-01", "end_date": "2025-05-08"}</function>\n\nThis call should return a list of transactions, but it seems that there was an issue with the previous call. Let me try to parse the data again.\n\n<function=aggregate_transactions>{"records": "[{\'txn_date\': \'2025-05-01\', \'merchant\': \'Uber\', \'amt\': 242.5}]", "agg": "sum"}</function>\n\nUnfortunately, I still get an error. This could be due to the structure of the data. Let me try to parse it again.\n\n<function=aggregate_transactions>{"records": "[{\'txn_date\': \'2025-05-01\', \'merchant\': \'Uber\', \'amt\': 242.5}]", "agg": "sum"}; re

In [51]:
tools = {"get_transactions": get_transactions}
tools_schemas = [t.tool_schema for t in tools.values()]
print(tools)

{'get_transactions': <function get_transactions at 0x000002276FD1ACA0>}


In [52]:
response = model.chat( message, tools=tools_schemas)

In [56]:
tool_calls = response.tool_calls if response.tool_calls else None

if tool_calls:
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = tool_call.function.arguments
        tool_result = tools[tool_name](tool_args)
        print(tool_result)



TypeError: print() argument after ** must be a mapping, not str

In [ ]:
import datetime

def get_chat_history(thread_id):
    history = memory.get_thread_history(thread_id)
    return history if history else []


def call_model(context):
    messages = context["messages"]
    user_input = messages[-1]["content"]

    system_prompt = f"""
                    Today is {datetime.datetime.now().strftime('%d %B %Y,  %I:%M %p')}.
                    {SYSTEM_PROMPT}
                    """

    full_messages = [{"role": "system", "content": system_prompt}] + messages

    response = client.chat.completions.create(
        model=model_name,
        messages=full_messages,
        temperature=0.6,
        tools=[t.tool_schema for t in tools.values()]
    )

    reply = response.choices[0].message
    context["messages"].append(reply)
    
    if reply.content and ("<tool_call" in reply.content or "<function" in reply.content):
        context["messages"].append({
            "role": "assistant",
            "content": "Error: Detected invalid manual tool call formatting. Please try rephrasing."
            })
        return context

    return context

In [ ]:
def should_continue(context):
    last_msg = context["messages"][-1]
    print("Last Message: ", last_msg)
    if isinstance(last_msg, dict) and last_msg["role"] == "function":
        return "call_model"
    if last_msg.tool_calls:
        return "tools"
    if last_msg.content:
        return "end"
    return "end"

In [ ]:
def run(self, user_input, thread_id):
    history = self.get_chat_history(thread_id)

    context = {
        "messages": [],
        "state": "call_model",
        "thread_id": thread_id,
    }

    context["messages"].extend(history)

    context["messages"].append({"role": "user", "content": user_input})

    while context["state"] != "end":
        if context["state"] == "call_model":
            context = self.call_model(context)
        elif context["state"] == "tools":
            context = self.tool_node(context)
        else:
            break

        context["state"] = self.should_continue(context)

    last_msg = context["messages"][-1]

    clean_messages = [
        {"role": "user", "content": user_input},
        {"role": last_msg.role,"content": last_msg.content.replace(SYSTEM_PROMPT, "")}
        ]

    self.memory.save_thread_history(thread_id, clean_messages)

    print("History: ", self.memory.get_thread_history(thread_id))

    return context["messages"][-1].content
